# Análisis de correlación

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_excel('https://github.com/adan-rs/AnalisisDatos/raw/main/data/enigh2020.xlsx')

Eliminemos datos atípicos

In [ ]:
from sklearn.ensemble import IsolationForest

def remove_outliers_iso_forest(df, columns, contamination=0.05, random_state=42):
    """
    Elimina valores atípicos utilizando el algoritmo Isolation Forest.
    Retorna DataFrame sin valores atípicos.
    """
    # Initialize and fit Isolation Forest model
    iso_forest = IsolationForest(contamination=contamination, random_state=random_state)
    iso_forest.fit(df[columns])
    
    # Predict labels: 1 (normal) or -1 (outlier)
    labels = iso_forest.predict(df[columns])
    
    # Calculate and display number of excluded outliers
    df_clean = df[labels == 1]
    excluded_values = len(df) - len(df_clean)
    print(f"\nCantidad de valores atípicos excluidos: {excluded_values}")
    return df_clean

In [ ]:
df = remove_outliers_iso_forest(df, ['ing_cor', 'gasto_mon'])

In [ ]:
df.plot(kind='scatter', x='ing_cor', y='gasto_mon');

In [ ]:
# Calcula la correlación entre "ing_cor" y "gasto_mon"
correlacion = df['ing_cor'].corr(df['gasto_mon'])
correlacion.round(4)

## Prueba de hipótesis para la correlación

*¿Para qué se utiliza?*
Esta prueba se emplea para determinar si existe una relación lineal significativa entre dos variables cuantitativas. Evalúa si el coeficiente de correlación de Pearson (r) —que mide la fuerza y dirección de una relación lineal— es estadísticamente distinto de cero en la población.

Algunos ejemplos de uso en el área de negocios son:
- Un analista financiero quiere investigar si existe una relación entre la tasa de interés interbancaria y el precio promedio mensual de las acciones de una empresa del sector consumo.
- El departamento de marketing desea saber si existe una relación directa entre el gasto mensual en publicidad digital y las ventas mensuales de un producto.

*Variables consideradas:*
Requiere dos variables cuantitativas medidas en escala de intervalo o razón. Ejemplos incluyen edad e ingresos, temperatura y consumo eléctrico, horas de estudio y calificación.

*¿Cómo funciona?*
La prueba calcula el valor del coeficiente r, que varía entre -1 y 1:
- r > 0: correlación positiva (a mayor valor de una variable, mayor valor de la otra).
- r < 0: correlación negativa (a mayor valor de una variable, menor valor de la otra).
- r = 0: ausencia de relación lineal.

Luego, se contrasta si este valor es significativamente distinto de cero mediante una prueba t con n - 2 grados de libertad.

*Hipótesis planteadas:*
- Hipótesis nula (H₀): No existe correlación entre las variables (r = 0).
- Hipótesis alternativa (H₁): Existe una correlación distinta de cero (r ≠ 0).

También puede realizarse un contraste unilateral si el objetivo es evaluar solo si la correlación es positiva o negativa.

*Supuestos o requisitos principales:*
- Linealidad: La relación entre las dos variables debe ser aproximadamente lineal. Esto puede verificarse visualmente con un diagrama de dispersión.
- Normalidad: Ambas variables deben seguir una distribución normal o, en su defecto, la muestra debe ser lo suficientemente grande (n > 30) para aplicar el teorema del límite central.
- Homogeneidad de la varianza: Se espera que la variabilidad de una variable sea similar a lo largo del rango de la otra (homocedasticidad).
- Independencia: Las observaciones deben ser independientes entre sí.

*Criterio de decisión:*
Si el valor p asociado al estadístico t es menor que el nivel de significancia (por ejemplo, α = 0.05), se rechaza la hipótesis nula, concluyendo que existe una relación lineal estadísticamente significativa entre las variables.

*Procedimiento*. Se recomienda utilizar la biblioteca *scipy.stats* debido a que permite obtener el valor p. Para ello se puede usar:

```
corr_coef, p_valor = pearsonr(df['X'], df['Y'])  
print('Coeficiente de correlación: ', corr_coef)  
print('Valor p', p_valor)
```

In [ ]:
from scipy.stats import pearsonr

def prueba_correlacion_pearson(df, var1, var2, alfa=0.05, imprimir=True):
    """
    Realiza una prueba de correlación de Pearson entre dos variables numéricas.
    Retorna un diccionario con los resultados de la prueba.
    """
    # Verificar condiciones de aplicabilidad
    if var1 not in df.columns or var2 not in df.columns:
        raise ValueError(f"Una o ambas variables no existen en el DataFrame: {var1}, {var2}")    
    if not np.issubdtype(df[var1].dtype, np.number) or not np.issubdtype(df[var2].dtype, np.number):
        raise ValueError("Ambas variables deben ser numéricas para calcular la correlación de Pearson")
    datos_completos = df[[var1, var2]].dropna()
    
    # Realizar prueba de correlación
    correlacion, p_valor = pearsonr(datos_completos[var1], datos_completos[var2])
        
    # Determinar resultado
    resultado = ('Existe una correlación significativa entre ambas variables' 
                 if p_valor < alfa 
                 else 'No existe una correlación significativa entre ambas variables')

    # Interpretar la fuerza de la correlación
    interpretacion = ("Correlación muy fuerte" if abs(correlacion) >= 0.9 else
                      "Correlación fuerte" if abs(correlacion) >= 0.7 else
                      "Correlación moderada" if abs(correlacion) >= 0.5 else
                      "Correlación débil" if abs(correlacion) >= 0.3 else
                      "Correlación nula o muy débil")
    interpretacion += " (negativa)" if correlacion < 0 else " (positiva)" if correlacion > 0 else ""

    # Imprimir resultados
    print(f'\nCorrelación entre {var1} y {var2}:')
    print(f'Coeficiente de correlación: {correlacion:.4f}')
    print(f'Valor p: {p_valor:.4f}')
    print(f'Interpretación: {interpretacion}')
    print(f'Resultado: {resultado}')

    # Retornar p-valor
    return p_valor

In [ ]:
resultados = prueba_correlacion_pearson(df, 'ing_cor', 'gasto_mon')

Ejemplo de reporte de metodología y resultados:
> “Se realizó una prueba de correlación de Pearson para evaluar la relación entre el ingreso corriente y el gasto monetario. Se encontró una correlación positiva significativa entre ambas variables (r = 0.6354. p < 0.001)”

## Alternativa
El coeficiente de correlación de Spearman es una alternativa apropiada cuando las variables son ordinales, existen relaciones monótonas no lineales, o la distribución no es normal. 
```
from scipy.stats import spearmanr
corr_coef, p_value = spearmanr(x, y)
```

In [ ]:
from scipy.stats import spearmanr

def prueba_correlacion_spearman(df, var1, var2, alfa=0.05):
    """
    Realiza una prueba de correlación de Spearman entre dos variables.
    Retorna un diccionario con los resultados de la prueba.
    """
    # Verificar condiciones de aplicabilidad
    if var1 not in df.columns or var2 not in df.columns:
        raise ValueError(f"Una o ambas variables no existen en el DataFrame: {var1}, {var2}")
    datos_completos = df[[var1, var2]].dropna()
    
    # Realizar prueba de correlación
    correlacion, p_valor = spearmanr(datos_completos[var1], datos_completos[var2])
    
    # Determinar resultado e interpretación
    resultado = ('Existe una correlación significativa entre ambas variables' 
                if p_valor < alfa 
                else 'No existe una correlación significativa entre ambas variables')
    
    interpretacion = ("Correlación muy fuerte" if abs(correlacion) >= 0.9 else
                     "Correlación fuerte" if abs(correlacion) >= 0.7 else
                     "Correlación moderada" if abs(correlacion) >= 0.5 else
                     "Correlación débil" if abs(correlacion) >= 0.3 else
                     "Correlación nula o muy débil")
    interpretacion += " (negativa)" if correlacion < 0 else " (positiva)" if correlacion > 0 else ""
    
    # Imprimir resultados
    print(f'\nCorrelación de Spearman entre {var1} y {var2}:')
    print(f'Coeficiente de correlación: {correlacion:.4f}')
    print(f'Valor p: {p_valor:.4f}')
    print(f'Interpretación: {interpretacion}')
    print(f'Resultado: {resultado}')
    
    return p_valor

In [ ]:
resultados = prueba_correlacion_spearman(df, 'ing_cor', 'gasto_mon')

## Ejercicio

Utilizando el mismo dataframe del ejemplo, realiza una prueba de hipótesis para la correlación para evaluar si existe una relación significativa entre el ingreso y alguna de las variables de gasto.

In [ ]:
df.info()

## Lecturas recomendadas

- El caso del "datasaurio" ilustra bien la importancia de graficar las variables: https://www.scientificamerican.com/article/what-this-graph-of-a-dinosaur-can-teach-us-about-doing-better-science/
- Para una discusión de cómo tomar decisiones a partir de correlaciones es recomendable la siguiente lectura: https://hbr.org/2014/03/when-to-act-on-a-correlation-and-when-not-to